In [8]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
from scipy import stats
import pandas as pd
from config import apikey, email

In [10]:
# base url for state codes needed down the line to focus in 
state_url = (f'https://aqs.epa.gov/data/api/list/states?email={email}&key={apikey}')

# get the state codes into a data frame
response=requests.get(state_url)
data=response.json()

# print to practice getting values
state_df=data['Data'][0]['code']
print(f'{}')
# 


ValueError: arrays must all be same length

In [6]:
data

{'Header': [{'status': 'Success',
   'request_time': '2021-02-02T20:59:24-05:00',
   'url': 'https://aqs.epa.gov/data/api/list/states?email=erich.mitchell87@gmail.com&key=goldmouse56',
   'rows': 56}],
 'Data': [{'code': '01', 'value_represented': 'Alabama'},
  {'code': '02', 'value_represented': 'Alaska'},
  {'code': '04', 'value_represented': 'Arizona'},
  {'code': '05', 'value_represented': 'Arkansas'},
  {'code': '06', 'value_represented': 'California'},
  {'code': '08', 'value_represented': 'Colorado'},
  {'code': '09', 'value_represented': 'Connecticut'},
  {'code': '10', 'value_represented': 'Delaware'},
  {'code': '11', 'value_represented': 'District Of Columbia'},
  {'code': '12', 'value_represented': 'Florida'},
  {'code': '13', 'value_represented': 'Georgia'},
  {'code': '15', 'value_represented': 'Hawaii'},
  {'code': '16', 'value_represented': 'Idaho'},
  {'code': '17', 'value_represented': 'Illinois'},
  {'code': '18', 'value_represented': 'Indiana'},
  {'code': '19', 'va